# Декомпозиция задач

## Обзор

Это учебное пособие посвящено концепции декомпозиции задач в инженерии промптов, с акцентом на техники разделения сложных задач на подзадачи и последовательного объединения подзадач в промптах. Эти методы необходимы для эффективного использования крупных языковых моделей для решения многошаговых задач и проведения сложных рассуждений.

## Мотивация

По мере развития языковых моделей ИИ они всё лучше справляются со сложными задачами. Однако такие модели часто работают эффективнее, если получают чёткие, поэтапные инструкции. Декомпозиция задач — мощный приём, позволяющий разбивать сложные проблемы на более мелкие, управляемые подзадачи. Такой подход не только улучшает работу модели, но и повышает интерпретируемость и надёжность получаемых результатов.

## Ключевые компоненты

1. **Декомпозиция сложных задач:** Техники анализа и разделения сложных задач на более простые подзадачи.
2. **Цепочки подзадач:** Методы последовательного объединения нескольких подзадач для решения более крупной задачи.
3. **Проектирование промптов для подзадач:** Создание эффективных промптов для каждой декомпозированной подзадачи.
4. **Интеграция результатов:** Объединение результатов отдельных подзадач для формирования комплексного решения.

---

## Установка окружения

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

from dotenv import load_dotenv
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('BASE_URL')

llm = ChatOpenAI(model="gpt-4o-mini", base_url=base_url)

def run_prompt(prompt, **kwargs):
    """Helper function to run a prompt through the language model.

    Args:
        prompt (str): The prompt template string.
        **kwargs: Keyword arguments to fill the prompt template.

    Returns:
        str: The model's response.
    """
    prompt_template = PromptTemplate(template=prompt, input_variables=list(kwargs.keys()))
    chain = prompt_template | llm
    return chain.invoke(kwargs).content

## Декомпозиция сложных задач

Давайте начнем со сложной задачи и разложим её на подзадачи. В качестве примера будем использовать анализ учебной успеваемости класса.

In [ ]:
complex_task = """
Проанализируйте учебную успеваемость класса по следующим данным:
- Количество учеников: 30
- Средний балл по математике: 3.8 из 5
- Количество учеников с низкой успеваемостью (оценка ниже 3): 6
- Количество олимпиадников: 4
- Среднее количество домашних заданий, выполненных вовремя: 85%
"""

decomposition_prompt = """
Разбейте задачу анализа учебной успеваемости класса на 3 подзадачи. Для каждой подзадачи кратко опишите, что она должна включать.

Задача: {task}

Подзадачи:
1.
"""

subtasks = run_prompt(decomposition_prompt, task=complex_task)
print(subtasks)

## Цепочка подзадач в промптах

Теперь, когда подзадачи определены, создадим отдельный промпт для каждой и соединим их в цепочку.

In [ ]:
def analyze_grades(avg_score, low_performers, total_students):
    """Подзадача 1: анализ успеваемости по оценкам.

    Args:
        avg_score (float): Средний балл по предмету.
        low_performers (int): Количество учеников с оценкой ниже 3.
        total_students (int): Общее количество учеников в классе.

    Returns:
        str: Анализ успеваемости.
    """
    prompt = """
    Проанализируй успеваемость класса по математике на основе данных:
    - Количество учеников: {total_students}
    - Средний балл: {avg_score} из 5
    - Учеников с оценкой ниже 3: {low_performers}

    Рассчитай долю отстающих, оцени средний балл и дай краткий вывод об уровне успеваемости.
    """
    return run_prompt(prompt, avg_score=avg_score, low_performers=low_performers, total_students=total_students)

def analyze_engagement(homework_rate, olympiad_count, total_students):
    """Подзадача 2: анализ вовлечённости учеников.

    Args:
        homework_rate (int): Процент домашних заданий, сданных вовремя.
        olympiad_count (int): Количество олимпиадников.
        total_students (int): Общее количество учеников в классе.

    Returns:
        str: Анализ вовлечённости.
    """
    prompt = """
    Проанализируй вовлечённость учеников класса на основе данных:
    - Количество учеников: {total_students}
    - Домашние задания сданы вовремя: {homework_rate}%
    - Количество участников олимпиад: {olympiad_count}

    Оцени уровень дисциплины и активности класса. Дай краткий вывод.
    """
    return run_prompt(prompt, homework_rate=homework_rate, olympiad_count=olympiad_count, total_students=total_students)

def generate_recommendations(low_performers, homework_rate):
    """Подзадача 3: рекомендации по улучшению.

    Args:
        low_performers (int): Количество отстающих учеников.
        homework_rate (int): Процент своевременно сданных домашних заданий.

    Returns:
        str: Рекомендации для учителя.
    """
    prompt = """
    На основе данных об успеваемости класса:
    - Количество учеников с низкой успеваемостью: {low_performers}
    - Процент домашних заданий, сданных вовремя: {homework_rate}%

    Предложи 3–4 конкретные рекомендации для учителя, как повысить успеваемость и вовлечённость.
    """
    return run_prompt(prompt, low_performers=low_performers, homework_rate=homework_rate)

# Запуск цепочки подзадач
grades_analysis = analyze_grades(3.8, 6, 30)
engagement_analysis = analyze_engagement(85, 4, 30)
recommendations = generate_recommendations(6, 85)

print("Анализ успеваемости:\n", grades_analysis)
print("\nАнализ вовлечённости:\n", engagement_analysis)
print("\nРекомендации:\n", recommendations)

## Интеграция результатов

Теперь объединим результаты всех подзадач, чтобы получить общий анализ успеваемости класса.

In [ ]:
def integrate_results(grades, engagement, recommendations):
    """Объединение результатов подзадач в общий анализ.

    Args:
        grades (str): Анализ успеваемости.
        engagement (str): Анализ вовлечённости.
        recommendations (str): Рекомендации.

    Returns:
        str: Общий анализ состояния класса.
    """
    prompt = """
    На основе следующих анализов составь общую оценку состояния класса:

    Анализ успеваемости:
    {grades}

    Анализ вовлечённости:
    {engagement}

    Рекомендации:
    {recommendations}

    Кратко подведи итоги и дай общую оценку ситуации в классе.
    """
    return run_prompt(prompt, grades=grades, engagement=engagement, recommendations=recommendations)

overall_analysis = integrate_results(grades_analysis, engagement_analysis, recommendations)
print("Общий анализ состояния класса:\n", overall_analysis)

## Заключение

По итогу этого урока вы освоили:
- Анализ сложной задачи и её разбиение на управляемые подзадачи.
- Создание отдельного промпта для каждой подзадачи.
- Объединение промптов в цепочку для многошагового рассуждения модели.
- Интеграцию результатов нескольких подзадач в общий вывод.

Эти навыки позволяют эффективнее использовать языковые модели для решения сложных задач и повышают качество и надёжность результатов.